In [11]:
SOURCE_STRING = """
Advanced Voice Model Release

OpenAI is rolling out an advanced voice model for ChatGPT Plus and Team users over the course of a week.
"""




In [12]:
STORYBOARD_PROMPT = """
You are a helpful assistant that takes a source string of news and exciting events, and returns a storyboard for a short video with content about the news. 
"""


In [17]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI()


ACTION_DESCRIPTION = """
Action Guidelines:

Keep the actions simple and concise. No more than 2 sentences.


1. **Be Specific:** Clearly describe the main subject, setting, and any key elements you want in the video.
    - Example: "A serene beach at sunset with waves gently crashing on the shore and seagulls flying in the sky."
2. **Include Details:** Mention any particular details that are important for the video.
    - Example: "The beach has golden sand and a few palm trees. The sky is painted with hues of pink and orange."
3. **Focus on Emotions or Atmosphere:** Describe the mood or atmosphere you want to convey.
    - Example: "The video should feel peaceful and calming.”
4. **Use Simple Language:** Avoid complex terms and jargon. Simple, straightforward language is most effective.
    - Example: "A cozy living room with a crackling fireplace and a cat sleeping on the rug."

By following these steps, you can create effective prompts that help the AI generate high-quality videos that align with your vision.
"""

class Storyboard(BaseModel):
    start_image_description: str = Field(..., description="A description of the starting image. Make it representative of the news.")
    # end_image_description: Optional[str] = Field(None, description="A description of the ending image, if it makes sense to animate to an image")
    action: str = Field(..., description=ACTION_DESCRIPTION)

# Call OpenAI API with GPT-4 Turbo
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[  
        {"role": "system", "content": STORYBOARD_PROMPT},
        {"role": "user", "content": SOURCE_STRING}
    ],
    response_format=Storyboard
)

# Extract the generated storyboard
storyboard = response.choices[0].message.parsed

print("Starting Image Description:")
print(storyboard.start_image_description)

# if storyboard.end_image_description:
#     print("Ending Image Description:")
#     print(storyboard.end_image_description)

print("Action:")
print(storyboard.action)



Starting Image Description:
A vibrant tech office with engineers working on computers, showcasing screens with code, voice waveforms, and the OpenAI logo prominently displayed.
Action:
The video starts with a busy tech office where engineers are collaborating on the new voice model. As they discuss, visuals of voice waveforms animate alongside text detailing the features and benefits of the advanced voice model.


In [18]:
%reload_ext autoreload
%autoreload 2

import requests
IDEOGRAM_URL = "https://api.ideogram.ai/generate"

IDEOGRAM_HEADERS = {
    "Api-Key": os.getenv("IDEOGRAM_API_KEY"),
    "Content-Type": "application/json"
}
start_image_request = {
    "image_request": {
        "prompt": storyboard.start_image_description,
        "model": "V_2",
        "magic_prompt_option": "AUTO"
    }
}


start_image_response = requests.post(IDEOGRAM_URL, json=start_image_request, headers=IDEOGRAM_HEADERS)
print(start_image_response.json())

# if storyboard.end_image_description:
#     end_image_request = {
#         "image_request": {
#             "prompt": storyboard.end_image_description,
#         "model": "V_2",
#         "magic_prompt_option": "AUTO"
#     }   
# }

#     end_image_response = requests.post(IDEOGRAM_URL, json=end_image_request, headers=IDEOGRAM_HEADERS)
#     print(end_image_response.json())



{'created': '2024-09-27T04:42:48.559185+00:00', 'data': [{'is_image_safe': True, 'prompt': 'A photo of a tech office with several engineers working at their desks. The engineers are typing on their keyboards and are intently focused on their screens. The screens display various code, voice waveforms, and the OpenAI logo. The office has a modern design with a mix of wooden and metal furniture, green plants, and hanging lights.', 'resolution': '1024x1024', 'seed': 1685841975, 'style_type': 'GENERAL', 'url': 'https://ideogram.ai/api/images/ephemeral/5occ11irQ9S8EVsLeYAjrQ.png?exp=1727498588&sig=041946d7909ba21afb8f9331d295a6138f9e5f76f5c7d962e53c1446cb3e967d'}]}


In [19]:
from lumaai import LumaAI

client = LumaAI()

generation = client.generations.create(
    prompt=storyboard.action,
    keyframes={
      "frame0": {
        "type": "image",
        "url": start_image_response.json()["data"][0]["url"]
      },
      
      # "frame1": {
      #   "type": "image",
      #   "url": end_image_response.json()["data"][0]["url"]
      # } if storyboard.end_image_description else None
    },
    loop=True,
)

print("Generation started for prompt: ", storyboard.action)
print(generation.id)

Generation started for prompt:  The video starts with a busy tech office where engineers are collaborating on the new voice model. As they discuss, visuals of voice waveforms animate alongside text detailing the features and benefits of the advanced voice model.
dc4a3ff7-5cae-4d94-b126-756623463669


In [20]:
import time

def poll_generation(generation_id, max_attempts=30, delay=10):
    start_time = time.time()
    for attempt in range(max_attempts):
        status = client.generations.get(generation_id)
        if status.state == "completed":
            end_time = time.time()
            return status, end_time - start_time
        elif status.state == "failed":
            raise Exception(f"Generation failed: {status.failure_reason}")
        print(f"Generation in progress... (Attempt {attempt + 1}/{max_attempts})")
        time.sleep(delay)
    raise Exception("Generation timed out")

try:
    start_time = time.time()
    result, generation_time = poll_generation(generation.id)
    print(f"Generation completed successfully in {generation_time:.2f} seconds!")
    print(f"Result URL: {result.assets}")
except Exception as e:
    print(f"Error: {str(e)}")


Generation in progress... (Attempt 1/30)
Generation in progress... (Attempt 2/30)
Generation in progress... (Attempt 3/30)
Generation in progress... (Attempt 4/30)
Generation in progress... (Attempt 5/30)
Generation in progress... (Attempt 6/30)
Generation in progress... (Attempt 7/30)
Generation in progress... (Attempt 8/30)
Generation completed successfully in 83.03 seconds!
Result URL: Assets(video='https://storage.cdn-luma.com/dream_machine/596e160c-4954-427e-ae7d-9a65068f039c/68043563-4d6d-4018-8ed2-1f586f305132_raw_video_0.mp4_video0668a588ccbea4b289c306e6d1c0c1cfc.mp4')
